# 1. Identificación del problema:

*Contexto* En mi trabajo como profesor de inglés, encuentro 2 tareas terriblemente tediosas que consumen tiempo que no tengo: 
Preparar nuevos contenidos y corregir tareas.
El feedback para mis estudiantes es una parte fundamental para mejorar sus capacidades en inglés, por lo que vamos a crear una herramienta que solucione el problema 2.
*Frecuencia*: Esta tarea la realizo de manera semanal, por lo que si esto tiene un buen resultado, puede ayudarme ahorrando tiempo de trabajo FUERA del instituto, dándome tiempo libre para hacer actividades o terminar mi bootcamp

*Intentos previos*: Mis intentos previos han funcionado, pasándole a la API de Chatgpt estos archivos, de manera que tardo un poco más, aunque también creo que esto me lleva a revisar más en profundidad (es fácil comprobar errores, o ver problemas al puntuar. Además, no siempre los estudiantes entregan la tarea en el formato pedido, o suben una foto con los ejercicios hechos a cuaderno)


# 2. Planificación de la solución

*Capacidades del LLM que aprovecharemos*:
- Lectura y comprensión de texto
- Razonamiento y toma de decisiones
- Generación de texto 

*Flujo de mi solución*

Dado que no tenemos un API key que nos permita integrar Moodle con nuestra herramienta, la idea es la siguiente: crear una carpeta donde añadamos todos los archivos de los estudiantes, crear un workflow con el que :

- Demos la tarea y el nivel a nuestro agente.
- especifiquemos la ruta para que nuestro agente lea todos estos documentos
- Antes de empezar a evaluar, que el agente vea el nivel general de la clase para ajustar las puntuaciones 
- Genere un documento final por alumno
<pre>
┌──────────────────────────┐
│  Configuración inicial   │
│  - Ruta de la tarea      │
│  - Nivel / rúbrica       │
└─────────────┬────────────┘
              │
┌─────────────▼────────────┐
│  Carga de trabajos       │
│  (DOCX desde carpeta)    │
└─────────────┬────────────┘
              │
┌─────────────▼────────────┐
│  Construcción del estado │
│  - Todos los trabajos    │
│  - Rúbrica               │
└─────────────┬────────────┘
              │
┌─────────────▼────────────┐
│  Análisis global de la   │
│  clase (LLM)             │
│                          │
│  - Nivel medio           │
│  - Errores comunes       │
│  - Aspectos logrados     │
│  - Diferencias clave     │
└─────────────┬────────────┘
              │
      ┌───────▼────────┐
      │ Persistencia   │
      │ feedback       │
      │ general        │
      │ (archivo)      │
      └───────┬────────┘
              │
┌─────────────▼────────────┐
│  Evaluación individual   │
│  con rúbrica y contexto  │
│  (por alumno)            │
│                          │
│  * Diseño paralelizable  │
│    (map / futuro)        │
└─────────────┬────────────┘
              │
┌─────────────▼────────────┐
│  Generación de archivos  │
│  por alumno              │
│  - Nota desglosada       │
│  - Feedback personalizado│
│  - Carpeta por tarea     │
└──────────────────────────┘
</pre>

    (hecho por chatgpt)

# 3. Selección de tecnología:

En este caso, creo que la mejor opción es utilizar LangGraph. De esta manera, puedo conseguir hacer procesos en paralelo, como hemos visto en el diagrama anterior. 
De esta manera, solucionamos dos problemas:
- Al tener un contexto del nivel general de la clase, 



# 4. Implementación de la solución
En este caso, vamos a abrir todos los archivos de tareas de estudiantes ()

In [5]:
#Instalamos las librerías necesarias
!pip install python-dotenv
!pip install google
!pip install wget
!pip install openai
!pip install --upgrade pip
!pip install openai-client
!pip install getpass4
!pip install python-docx
!pip install -q langgraph langchain langchain-openai
!apt-get install -y graphviz graphviz-dev > /dev/null 2>&1
!pip install -q pygraphviz


ERROR: Could not find a version that satisfies the requirement openai-client (from versions: none)
ERROR: No matching distribution found for openai-client


El sistema no puede encontrar la ruta especificada.
  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [88 lines of output]
      C:\Users\rammu\AppData\Local\Temp\pip-build-env-a3ae630a\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
        

In [36]:
import os
print(os.getenv("OPENAI_API_KEY"))
API_KEY = "YOUR_API_KEY"

None


In [7]:
# En esta celda, vamos a definir las funciones para cargar los trabajos y la rúbrica
# Abrimos todas las tareas de nuestros estudiantes, habilitando que se puedan abrir docs
# hacemos esto porque los pdf no son tan fáciles de manejar

from docx import Document
from pathlib import Path



#creamos la función para cargar los trabajos
def cargar_trabajos_docx(ruta: str | Path) -> dict[str, str]:
    """
    Lee todos los archivos .docx de una carpeta y devuelve
    un diccionario {nombre_archivo: texto}.
    """
    ruta = Path(ruta)
    trabajos = {}

    for archivo in ruta.glob("*.docx"):
        doc = Document(archivo)
        texto = "\n".join(p.text for p in doc.paragraphs)
        trabajos[archivo.stem] = texto

    return trabajos
#creamos la función para cargar la rúbrica

def cargar_rubrica_docx(ruta: str | Path) -> str:
    """
    Lee un archivo .docx de rúbrica y devuelve su contenido como texto.
    """
    ruta = Path(ruta)

    if not ruta.exists():
        raise FileNotFoundError(f"No existe la rúbrica en: {ruta}")

    doc = Document(ruta)
    texto = "\n".join(p.text for p in doc.paragraphs)

    return texto




In [8]:
#establecemos las rutas y cargamos los trabajos y la rúbrica
from pathlib import Path

RUTA_TAREAS = Path("tareas/Tarea 1")
NOMBRE_TAREA = RUTA_TAREAS.name
trabajos = cargar_trabajos_docx(RUTA_TAREAS)
rubrica = cargar_rubrica_docx("tareas/Rúbricas/rubrica tarea 1.docx")

print(f"Trabajos cargados: {len(trabajos)}")

Trabajos cargados: 3


In [ ]:
# Configuramos el modelo de lenguaje

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    openai_api_key="sk-..."
)

In [18]:
# Definimos el estado de la clase usando TypedDict
from typing import TypedDict
class EstadoClase(TypedDict):
    trabajos: dict[str, str]
    rubrica: str
    nombre_tarea: str
    analisis_global: str | None
    evaluaciones: dict[str, str] | None

In [19]:
# Implementamos el nodo de análisis global
from pathlib import Path

def nodo_analisis_global(state: EstadoClase) -> EstadoClase:
    trabajos_texto = ""
    for alumno, texto in state["trabajos"].items():
        trabajos_texto += f"\n---\n{alumno}:\n{texto}"

    prompt = f"""
    Eres un profesor que analiza el nivel general de una clase.

    Analiza:
    - Nivel medio del grupo
    - Errores comunes
    - Aspectos bien trabajados
    - Qué diferencia a los mejores de los peores

    RÚBRICA:
    {state['rubrica']}

    TRABAJOS:
    {trabajos_texto}

    NO evalúes alumnos individualmente.
    Haz un análisis global del grupo.

    Proporciona el análisis de forma clara y estructurada.
    Incluye: Una lista de errores comunes que puedan tener los estudiantes y que puedan ser evaluados
    en la asignatura de inglés. (Ejemplo: ¿Hay un error gramatical que se repite? ¿Una palabra mal usada? ¿Construcciones gramaticales calcadas del español?)
    Para cada uno, incluye ejemplos extraídos de los trabajos de los estudiantes, mostrando el error y una posible solución o corrección.
    No incluyas sólo 3 errores, incluye tantos como creas que se repiten de manera significativa, haciendo que el análisis sea útil para mejorar la enseñanza.
    Ten en cuenta que mi intención es usar este análisis para hacer un PowerPoint que me ayude a explicar a los estudiantes en qué deben mejorar, a nivel gramatical, de vocabulario y de estilo de redacción.
    """

    respuesta = llm.invoke(prompt)
    analisis = respuesta.content

    carpeta = Path("evaluaciones") / state["nombre_tarea"]
    carpeta.mkdir(parents=True, exist_ok=True)

    archivo_feedback = carpeta / "feedback_general.txt"
    archivo_feedback.write_text(analisis, encoding="utf-8")

    return {
        **state,
        "analisis_global": analisis
    }


In [20]:
# Implementamos la función para parsear JSON de respuestas LLM. De esta forma, podemos extraer
# información estructurada de las respuestas del modelo de lenguaje.
import re
import json

def parsear_json_llm(texto: str) -> dict:
    """
    Extrae el primer bloque JSON válido de un texto.
    Lanza error si no encuentra ninguno.
    """
    match = re.search(r"\{.*\}", texto, re.DOTALL)
    if not match:
        raise ValueError("No se encontró un JSON válido en la respuesta")
    return json.loads(match.group())

In [28]:
# Implementamos el nodo de evaluación individual
# De esta forma, evaluamos cada trabajo individualmente

from pathlib import Path
def nodo_evaluacion_individual(state: EstadoClase) -> EstadoClase:
    evaluaciones = {}

    for alumno, texto in state["trabajos"].items():
        prompt = f"""
        Eres un profesor que evalúa trabajos de alumnos.

        ANÁLISIS GLOBAL DE LA CLASE:
        {state['analisis_global']}

        RÚBRICA:
        {state['rubrica']}

        TRABAJO DEL ALUMNO ({alumno}):
        {texto}
        Evalúa el trabajo del alumno según la rúbrica y el análisis global de la clase, tomando este como contexto para entender el nivel general y poner notas justas comparativamente

        Devuelve EXCLUSIVAMENTE un JSON con esta estructura exacta:

        {{
          "nota_final": número del 1 al 10,
          "criterios": {{
            "criterio_1": nombre del criterio, número, "comentario sobre criterio 1, explicando errores explícitos y añadiendo correcciones y cómo mejorar",
            "criterio_2": nombre del criterio, número, "comentario sobre criterio 2  explicando errores explícitos y añadiendo correcciones y cómo mejorar",
          }},
          "feedback": "comentario personalizado para el alumno"
        }}
        """

        respuesta = llm.invoke(prompt)

        evaluaciones[alumno] = respuesta.content

    return {
        **state,
        "evaluaciones": evaluaciones
    }


In [29]:
# Configuramos el flujo de trabajo con LangGraph
from langgraph.graph import StateGraph
graph = StateGraph(EstadoClase)

graph.add_node("analisis_global", nodo_analisis_global)
graph.add_node("evaluar_individual", nodo_evaluacion_individual)

graph.set_entry_point("analisis_global")
graph.add_edge("analisis_global", "evaluar_individual")
graph.set_finish_point("evaluar_individual")

workflow = graph.compile()

In [30]:
# Ejecutamos el flujo de trabajo

estado_inicial: EstadoClase = {
    "trabajos": trabajos,
    "rubrica": rubrica,
    "nombre_tarea": RUTA_TAREAS.name,
    "analisis_global": None,
    "evaluaciones": None
}

resultado = workflow.invoke(estado_inicial)

In [31]:
# Mostramos un ejemplo de evaluación
for alumno, evaluacion_json in resultado["evaluaciones"].items():
    print("ALUMNO:", alumno)
    print("TIPO:", type(evaluacion_json))
    print("CONTENIDO:")
    print(evaluacion_json)
    break

ALUMNO: alumno1
TIPO: <class 'str'>
CONTENIDO:
{
  "nota_final": 8.5,
  "criterios": {
    "criterio_1": "Intercultural content and awareness, 1.8, The student has shown a good understanding of the cultural differences between Spain and Portugal. The advice given is practical and useful, and the student has avoided stereotypes. However, the student could have included more reflection on the impact of these cultural differences on communication.",
    "criterio_2": "Practical usefulness of the guide, 1.4, The guide is generally useful and provides clear advice for Spanish speakers visiting Portugal. However, some of the advice is somewhat general and could be more specific. For example, the student could have provided more specific examples of words that have different meanings in Spanish and Portuguese.",
    "criterio_3": "Vocabulary range and appropriacy, 1.9, The student has used a wide range of vocabulary related to culture, communication and social interaction. There are a few min

In [32]:
# Guardamos las evaluaciones en archivos de texto individuales
import json
from pathlib import Path

salida = Path("evaluaciones")
salida.mkdir(exist_ok=True)

for alumno, evaluacion_json in resultado["evaluaciones"].items():
    datos = parsear_json_llm(evaluacion_json)

    contenido = f"""
    NOTA FINAL: {datos['nota_final']}

    DESGLOSE POR CRITERIOS:
    """

    for criterio, nota in datos["criterios"].items():
        contenido += f"- {criterio}: {nota}\n"

    contenido += f"\nFEEDBACK PERSONALIZADO:\n{datos['feedback']}\n"

    archivo = salida / f"{alumno}.txt"
    archivo.write_text(contenido, encoding="utf-8")


In [33]:
# Implementamos la función para guardar las evaluaciones en archivos de texto individuales
from pathlib import Path
import json

def guardar_evaluaciones(resultado, carpeta_base="evaluaciones"):
    nombre_tarea = resultado["nombre_tarea"]
    salida = Path(carpeta_base) / nombre_tarea
    salida.mkdir(parents=True, exist_ok=True)

    resultado_final = {}

    for alumno, evaluacion_json in resultado["evaluaciones"].items():
        datos = parsear_json_llm(evaluacion_json)
        resultado_final[alumno] = datos

        contenido = f"""
NOTA FINAL: {datos['nota_final']}

DESGLOSE POR CRITERIOS:
"""
        for criterio, nota in datos["criterios"].items():
            contenido += f"- {criterio}: {nota}\n"

        contenido += f"\nFEEDBACK PERSONALIZADO:\n{datos['feedback']}\n"

        archivo = salida / f"{alumno}.txt"
        archivo.write_text(contenido, encoding="utf-8")

    return resultado_final


In [34]:
# Guardamos las evaluaciones procesadas
resultado_final = guardar_evaluaciones(resultado)
resultado_final

{'alumno1': {'nota_final': 8.5,
  'criterios': {'criterio_1': 'Intercultural content and awareness, 1.8, The student has shown a good understanding of the cultural differences between Spain and Portugal. The advice given is practical and useful, and the student has avoided stereotypes. However, the student could have included more reflection on the impact of these cultural differences on communication.',
   'criterio_2': 'Practical usefulness of the guide, 1.4, The guide is generally useful and provides clear advice for Spanish speakers visiting Portugal. However, some of the advice is somewhat general and could be more specific. For example, the student could have provided more specific examples of words that have different meanings in Spanish and Portuguese.',
   'criterio_3': "Vocabulary range and appropriacy, 1.9, The student has used a wide range of vocabulary related to culture, communication and social interaction. There are a few minor inaccuracies, such as the incorrect use of

# 5. Evaluación y reflexión

Me gusta mucho cómo esta herramienta ha funcionado, y cómo, modificando algunas cositas dentro del prompt, y dándole una estructura clara, he conseguido una evaluación bastante buena para todo lo que mis alumnos necesitan. 


En este caso, he probado con una sóla tarea y distintos alumnos, pero me gustaría probar con más (Aunque no quiero gastar infinitos tokens)

la limitación de esta tarea, claramente, podría ser el formato de texto, por lo que implementar un sistema texto a voz para presentaciones podría ser interesante. 

Esta solución puede ayudarme a ahorrar mucho tiempo de mi trabajo, y utilizarlo exclusivamente para tareas que tengan que ver con ayudar a alumnos. Además, al darme un feedback con visión general, puedo utilizarla para entender los errores de mis alumnos y repasar contenidos que fallen de manera general. 

Creo que no ha "cambiado" mi percepción sobre las aplicaciones prácticas, pero sí que esta herramienta consigue automatizar el trabajo y eliminar muchos pasos intermedios (evita ir copiando y pegando cada tarea en chatgpt, sino que puedo descargar TODAS las entregas a la vez y pasárselas directamente)
Me ha parecido además muy interesante un resultado, y es que después de hacer distintas pruebas para ajustar el prompt, he visto que las notas se mantenían estables (siempre ha puntuado de la misma manera, aunque el feedback fuera distinto). Esto no me ha ocurrido cuando he utilizado la API de Chatgpt para evaluar tareas, por lo que entiendo que al haberle dado un contexto, la herramienta ha conseguido crear notas de una manera coherente. 
Sin embargo, aún estaría el problema de tener que copiar una a una cada evaluación. Además, he anonimizado cada tarea para evitar que openai tenga datos de alumnos, pero eso también supondría un problema ya que haría más difícil trabajar con esto (aunque codificar cada entrega y luego decodificarla no sería muy difícil

Sobre cómo he trabajado, comencé creando mi herramienta pensando en usar runnable.map para hacer un flujo paralelo. Sin embargo, vi que esto podría crear resultados con menos coherencia, por lo que opté por crear un sólo estado individual. Por eso, primero el LLM toma una idea general de cómo es el nivel del grupo, para así poder identificar errores comunes, y establece un marco de referencia
Teniendo este contexto, realiza las evaluaciones de forma secuencial e independiente. 


# 6. Escalabilidad:

- Crear funciones para evaluar otros formatos de texto y comprobar su validez (¿PDF? ¿OpenOffice? ¿.txt?)
- Explorar la posibilidad de automatizar esto aún más integrándolo con Moodle, la plataforma donde se entregan todas las tareas de la Comunidad de Madrid. Probablemente, con una API key podríamos descargar todas las tareas en el mismo directorio en lugar de hacerlo a mano, codificar los nombres de los alumnos (Un NER podría ser una idea) para eliminar cualquier marca dentro de la tarea o en el nombre. 
- De igual manera, podríamos, una vez creados todos los archivos de evaluación, pasarlos a moodle para que las notas se escriban automáticamente en el perfil del alumno, consiguiendo evitar un montón de trabajo innecesario

Visión a largo plazo:
Podría integrar esta solución aplicándolo a todos los aspectos de corrección (es decir, como dije antes, utilizar una función texto a voz para corregir presentaciones, o incluso, crear un nodo para corregir tareas de respuesta cerrada, como un solucionario)
Además, esta tarea podría servir para otros profesores añadiendo al estado un elemento llamado "asignatura" de manera que pudiera servir para corregir tareas de otras asignaturas como marketing o gestión de compra

Puerta abierta:
- Durante estas vacaciones de navidad, me gustaría explorar la opción de crear un nodo que evalúe si la asignatura es inglés o no: de esta manera, si el llm detecta que la asignatura es otra, debería cambiar la manera de evaluar y crear un nuevo prompt que pasar al nodo de evaluación individual y global.
- Además, quiero comprobar cuánto de difícil es hacer esta integración con Moodle
- tratar de implementar que acepte distintos formatos de texto